In [1]:
import cv2
import math
import os
import pytesseract
import re
import numpy as np
import csv

from PIL import Image, ImageFilter
from osgeo import gdal, osr
from tqdm import tqdm

In [2]:
def crop_map(input_path, output_path):
  # Load the JPEG file
  img = cv2.imread(input_path)

  # Convert the image to grayscale
  gray = cv2.cvtColor(
    img, 
    cv2.COLOR_BGR2GRAY
  )

  # Apply a Canny edge detection filter to find the edges
  edges = cv2.Canny(gray, 50, 150)

  # Find contours in the image
  contours, hierarchy = cv2.findContours(
    edges, 
    cv2.RETR_EXTERNAL, 
    cv2.CHAIN_APPROX_SIMPLE
  )

  # Find the largest contour
  largest_contour = max(
    contours, 
    key=cv2.contourArea
  )

  # Get the bounding box coordinates
  x, y, w, h = cv2.boundingRect(largest_contour)

  # Extract the image inside the bounding box
  img_cropped = img[y:y+h, x:x+w]

  # Save the cropped image as a new JPEG file
  cv2.imwrite(output_path, img_cropped)

In [3]:
def get_coordinates(input_path):
  # Read the PNG image using OpenCV
  image = cv2.imread(input_path)

  # Convert the image to grayscale
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  # Perform OCR to extract text from the image
  text = pytesseract.image_to_string(gray_image)
  
  lines = re.split(r'\s+|\n', text)

  matched_lines = [line for line in lines if line.startswith('X_min')]
  xmin = float(matched_lines[0].split('_')[-1])

  matched_lines = [line for line in lines if line.startswith('X_max')]
  xmax = float(matched_lines[0].split('_')[-1])

  matched_lines = [line for line in lines if line.startswith('Y_min')]
  ymin = float(matched_lines[0].split('_')[-1])

  matched_lines = [line for line in lines if line.startswith('Y_max')]
  ymax = float(matched_lines[0].split('_')[-1])
  
  return xmin, xmax, ymin, ymax

In [4]:
def georeference(input_path, output_path, xmin, xmax, ymin, ymax):
  # Load the input image
  input_image = gdal.Open(input_path)

  # Get the image dimensions
  image_width = input_image.RasterXSize
  image_height = input_image.RasterYSize
  num_bands = input_image.RasterCount

  # Create a spatial reference for the output GeoTIFF
  output_srs = osr.SpatialReference()
  output_srs.ImportFromEPSG(4326)  # Use WGS84 coordinate system

  # Create the output GeoTIFF file
  driver = gdal.GetDriverByName('GTiff')
  output_image = driver.Create(
    output_path, 
    image_width, 
    image_height, 
    num_bands, 
    gdal.GDT_Byte
  )

  # Write the output image data
  for band_index in range(1, num_bands + 1):
      input_band = input_image.GetRasterBand(band_index)
      output_band = output_image.GetRasterBand(band_index)
      output_band.WriteArray(input_band.ReadAsArray())

  # Set the output image geotransform
  geotransform = (
    xmin, 
    (xmax - xmin) / image_width, 
    0, 
    ymax, 
    0, 
    (ymin - ymax) / image_height
  )
  output_image.SetGeoTransform(geotransform)

  # Set the output image projection
  output_image.SetProjection(output_srs.ExportToWkt())

  # Close the input and output images
  input_image = None
  output_image = None

In [5]:
if not os.path.exists('/temp_file'):
  os.makedirs('/temp_file')
  
if not os.path.exists('/temp_file/straighten'):
  os.makedirs('/temp_file/straighten')

if not os.path.exists('/temp_file/crop'):
  os.makedirs('/temp_file/crop')

In [6]:
if not os.path.exists('georeferenced_maps'):
  os.makedirs('georeferenced_maps')

In [7]:
list_peta = os.listdir('scan_peta')

In [8]:
failed = []
for i in tqdm(list_peta):
  name = i.split('.')[0]
  
  try:
    xmin, xmax, ymin, ymax = get_coordinates(
      'scan_peta/{}.jpg'.format(name)
    )
    
    crop_map(
      'scan_peta/{}.jpg'.format(name),
      'temp_file/crop/{}.jpg'.format(name)
    )
    
    georeference(
      'temp_file/crop/{}.jpg'.format(name), 
      'georeferenced_maps/{}.tif'.format(name),
      xmin, xmax, ymin, ymax
    )
    
  except:
    failed.append(name)

with open("failed_georeference.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(failed)

  0%|                                                                                           | 0/22 [00:00<?, ?it/s]

scan_peta/Document_20230714_0010.jpg
118.69395973618504 118.69631715409326 -8.558126424747066 -8.555455249779108


  5%|███▊                                                                               | 1/22 [00:02<00:45,  2.14s/it]

scan_peta/Document_20230714_0011.jpg
118.68285338135313 118.68417081901835 -8.596279877836846 -8.59517388078456


  9%|███████▌                                                                           | 2/22 [00:05<01:00,  3.05s/it]

...

 91%|██████████████████████████████████████████████████████████████████████████▌       | 20/22 [00:53<00:05,  2.59s/it]

scan_peta/Document_20230714_0008.jpg


 95%|██████████████████████████████████████████████████████████████████████████████▎   | 21/22 [00:54<00:02,  2.21s/it]

scan_peta/Document_20230714_0009.jpg


100%|██████████████████████████████████████████████████████████████████████████████████| 22/22 [00:56<00:00,  2.57s/it]


In [28]:
list_peta

['Document_20230714_0010.jpg',
 'Document_20230714_0011.jpg',
 'Document_20230714_0012.jpg',
 'Document_20230714_0013.jpg',
 'Document_20230714_0014.jpg',
 'Document_20230714_0015.jpg',
 'Document_20230714_0016.jpg',
 'Document_20230714_0017.jpg',
 'Document_20230714_0018.jpg',
 'Document_20230714_0019.jpg',
 'Document_20230714_0020.jpg',
 'Document_20230714_0021.jpg',
 'Document_20230714_0022.jpg',
 'Document_20230714_0001.jpg',
 'Document_20230714_0002.jpg',
 'Document_20230714_0003.jpg',
 'Document_20230714_0004.jpg',
 'Document_20230714_0005.jpg',
 'Document_20230714_0006.jpg',
 'Document_20230714_0007.jpg',
 'Document_20230714_0008.jpg',
 'Document_20230714_0009.jpg']